# Exercise 1 - Get Started with Grounding

In this exercise, we'll start exploring the grounding on SAP AI Core. Grounding allows Large Language Models (LLMs) to reference specific knowledge sources when generating responses, which can help improve accuracy and relevance. 

We'll start with the basics by grounding an LLM using SAP.help as our knowledge source. 

This exercise will cover:

* Basic grounding concepts in SAP AI Core
* How to set up grounding with a document source
* Comparing responses with and without grounding


## Install SAP Cloud SDK for AI
For this workshop we will use the SAP Cloud SDK for AI : Python. SAP Cloud SDK for AI is the official SDFk for SAP AI core, generative AI Hub, and orchestration. Overall the SDK composed of three python disctribution:
* You can use **sap-ai-sdk-base** to access the AI API using Python methods and data structures.
* You can use **sap-ai-sdk-core** to interact with SAP AI Core for administration and public lifecycle management. We will need this later to create object store secrets. 
* You can use **sap-ai-sdk-gen** to integrate native SDK libraries and langchain for accessing models and leverage the orchestration service of generative AI hub with capabilities such as templating, grounding, data masking, and content filtering. For more information, see Generative AI Hub.



In [ ]:

! pip install sap-ai-sdk-gen
! pip install sap-ai-sdk-core 

## Prompt without Grounding 
> 🟨 **TODO:**  
> _Add Example_

## Ground your prompt with SAP Help
Let us get started with a quick form up by grounding you LLM with SAP Help. 
> 🟨 **TODO:**  
> _Add more description to each steps._

> 💡 **Tip:**  
> Restart Kernel

In [ ]:
import init_env
import variables

init_env.set_environment_variables()

#assert variables.RESOURCE_GROUP=='', """You should change the value assigned to the `RESOURCE_GROUP` in the `variables.py` file to your own resource group first!"""
print(f"Resource group is set to: {variables.RESOURCE_GROUP}")

Resource group is set to: AI167000


In [3]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.config import GroundingModule, OrchestrationConfig
from gen_ai_hub.orchestration.models.document_grounding import DocumentGrounding, DocumentGroundingFilter
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.service import OrchestrationService

llm = LLM(
    name="gemini-2.5-flash",
    parameters={
        'temperature': 0.0,
    }
)
template = Template(
            messages=[
                SystemMessage("You are a helpful translation assistant."),
                UserMessage("""Answer the request by providing relevant answers that fit to the request.
                Request: {{ ?user_query }}
                Context:{{ ?grounding_response }}
                """),
            ]
        )

In [4]:
# Set up Document Grounding
filters = [
            DocumentGroundingFilter(id="SAPHelp", data_repository_type="help.sap.com")
        ]

grounding_config = GroundingModule(
            type="document_grounding_service",
            config=DocumentGrounding(input_params=["user_query"], output_param="grounding_response", filters=filters)
        )

config = OrchestrationConfig(
    template=template,
    llm=llm,
    grounding=grounding_config
)

In [5]:
import importlib
variables = importlib.reload(variables)

orchestration_service = OrchestrationService(
    api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL,
    config=config
)

response = orchestration_service.run(
    template_values=[
        TemplateValue( 
            name="user_query",
            value="What roles do I need to access the SAP AI Launchpad ?"
        )
    ]
)

print(response.orchestration_result.choices[0].message.content)

To access the SAP AI Launchpad, you need to have the **required role collections assigned to your user**.

This is mentioned as a prerequisite when configuring and using SAP AI Launchpad in conjunction with SAP AI Core.

The other roles and IAM apps mentioned in the provided text (e.g., `S_ER9_11001966_QUICKCONFIG_TRAN`, `F7924_SUM_TRAN`, `F8552_TRAN`, `F8555_TRAN`, `SAP_CORE_BC_AIU_SUM_PC`, `SAP_CORE_BC_AIU_FIL_PC`, `SAP_CORE_BC_AIU_PER_PC`) are specific to enabling *generative AI features within SAP S/4HANA Cloud Public Edition* (like AI-assisted tax configuration, smart summarization, easy filter, or smart personalization) and are not for direct access to the SAP AI Launchpad application itself.


## Summary 
.....

Continue to - [Excercise 2: Ground your LLM with custom documents](ex2-1-vector-api.md)

> 🟨 **TODO:**  
> _Need to check whether use a direct Link to github.com._